# Predict the quality of Wine based on the description with custom Word Embeddings

## Load and explore data

In [21]:
import gensim.models
import gensim
import numpy as np
import pandas as pd
import re

df = pd.read_csv('data/winemag-data-130k-v2.csv')
df.head()


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [22]:
df.info()
df.points.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


88     17207
87     16933
90     15410
86     12600
89     12226
91     11359
92      9613
85      9530
93      6489
84      6480
94      3758
83      3025
82      1836
95      1535
81       692
96       523
80       397
97       229
98        77
99        33
100       19
Name: points, dtype: int64

In [23]:
# split points into binary label (80-89 = bad, 90-99 = good)
df['label'] = df['points'].apply(lambda x: 'good' if x > 89 else 'bad')

df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,label
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,bad
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,bad
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,bad
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,bad
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,bad


## Modelling

In [24]:
clean_txt = []

# TODO: find a better cleaning approach!
for w in range(len(df.description)):
    # make text lower case
    desc = df['description'][w].lower()

    #remove punctuation
    desc = re.sub('[^a-zA-Z]', ' ', desc)

    #remove tags
    desc = re.sub('&lt;/?.*?&gt;', ' &lt;&gt; ', desc)

    #remove digits and special chars
    desc = re.sub('(\\d|\\W)+', ' ', desc)
    clean_txt.append(desc)

df['clean_desc'] = clean_txt
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,label,clean_desc
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,bad,aromas include tropical fruit broom brimstone ...
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,bad,this is ripe and fruity a wine that is smooth ...
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,bad,tart and snappy the flavors of lime flesh and ...
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,bad,pineapple rind lemon pith and orange blossom s...
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,bad,much like the regular bottling from this comes...


In [25]:
corpus = []
for col in df.clean_desc:
    word_list = col.split(' ')
    corpus.append(word_list)

In [26]:
model = gensim.models.Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)
print(f'The word embedding has a vocabulary size of {len(model.wv)} words.')

model.save('embeddings\description_emb.bin')

The word embedding has a vocabulary size of 30463 words.


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

X = df['clean_desc']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def sentence_to_vector(sentence, word_embedding):
    vector = []
    for word in sentence.split():
        if word in word_embedding.wv:
            vector.append(word_embedding.wv[word])
    return np.mean(vector, axis=0)

def train_bernoulli(word_embedding_path):
    word_embedding = gensim.models.KeyedVectors.load(word_embedding_path)

    train_vectors = [sentence_to_vector(sentence, word_embedding) for sentence in X_train]
    test_vectors = [sentence_to_vector(sentence, word_embedding) for sentence in X_test]

    nb_model = BernoulliNB()
    nb_model.fit(train_vectors, y_train)

    y_pred = nb_model.predict(test_vectors)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')
    return nb_model

In [28]:
model_min1_vs100 = train_bernoulli('embeddings\description_emb.bin')

Accuracy: 0.693594922100404


So we have an accuracy of around 67%. This is worse than the tfidf vectorizer. We have multiple approaches to fix this. We could try to finetune the parameters of our word embedding, we could use other prebuilt word embeddings or we could also use other models than the BernoulliNB. Since the word embedding that we just built was rather simple we should start by improving on it first.

In [29]:
# Let's create word embeddings with different parameters

model_min_count_two_100 = gensim.models.Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=2, workers=4)
model_min_count_two_100.save('embeddings\description_emb_min2_vs100.bin')

model_min_count_three_100 = gensim.models.Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=3, workers=4)
model_min_count_three_100.save('embeddings\description_emb_min3_vs100.bin')

model_min_count_one_300 = gensim.models.Word2Vec(sentences=corpus, vector_size=300, window=5, min_count=1, workers=4)
model_min_count_one_300.save('embeddings\description_emb_min1_vs300.bin')

model_min_count_two_300 = gensim.models.Word2Vec(sentences=corpus, vector_size=300, window=5, min_count=2, workers=4)
model_min_count_two_300.save('embeddings\description_emb_min2_vs300.bin')

model_min_count_three_300 = gensim.models.Word2Vec(sentences=corpus, vector_size=300, window=5, min_count=3, workers=4)
model_min_count_three_300.save('embeddings\description_emb_min3_vs300.bin')

In [30]:
model_min2_vs100 = train_bernoulli('embeddings\description_emb_min2_vs100.bin')

Accuracy: 0.6837853433352568


In [31]:
model_min3_vs100 = train_bernoulli('embeddings\description_emb_min3_vs100.bin')

Accuracy: 0.6813618003462204


In [32]:
model_min1_vs300 = train_bernoulli('embeddings\description_emb_min1_vs300.bin')

Accuracy: 0.6836314675899211


In [33]:
model_min2_vs300 = train_bernoulli('embeddings\description_emb_min2_vs300.bin')

Accuracy: 0.6895556837853434


In [34]:
model_min3_vs300 = train_bernoulli('embeddings\description_emb_min3_vs300.bin')

Accuracy: 0.6786305058665127


The best performing word embedding seems to be the one with min_count=2 and vector_size=300. It's accuracy is around 69% which still is way worse than the tfidf vectorizer.
It seems like we cannot improve this any further with our own word embedding. Let's test some prebuilt word embeddings then.